<a href="https://colab.research.google.com/github/mashyko/NLP_BERT_Transformers/blob/master/gensim_fasttext_porchText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#gensim 、 fastText 、 Torchtext　を用いた単語間のコサイン類似度の計算

以下のウェブサイトを参考にしました。
https://qiita.com/MonaCat/items/c26fa660a711f010c39a

Torchtext　の使用については、『つくりながら学ぶ! PyTorchによる発展ディープラーニング」（小川雄太郎、マイナビ出版）』を参考にしています。

google drive のディレクトリ /data/に、学習済み日本語単語辞書/vector_neologd/と/tsv/ファイルが保存されている必要があります。
vedtor_neologdは以下からダウンロードできます。　https://drive.google.com/open?id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc

tsvファイルは各自で作成ください。このファイルの例は、上記の書籍にあります。

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

%cd /content/drive/My Drive/data
!ls

/content/drive/My Drive/data
t10k-images-idx3-ubyte	train-images-idx3-ubyte  tsv
t10k-labels-idx1-ubyte	train-labels-idx1-ubyte  vector_neologd


In [0]:
! pip install gensim

In [0]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('./vector_neologd/model.vec', binary=False)

model.most_similar(positive=['自然言語処理'])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('自然言語理解', 0.7600098848342896),
 ('自然言語', 0.7503659725189209),
 ('計算言語学', 0.7258570194244385),
 ('自動プログラミング', 0.6848069429397583),
 ('テキストマイニング', 0.6811494827270508),
 ('コンピュータ言語', 0.6618390083312988),
 ('メタプログラミング', 0.658093273639679),
 ('Webプログラミング', 0.6488876342773438),
 ('形態素解析', 0.6479052901268005),
 ('コーパス言語学', 0.6465639472007751)]

In [0]:
model.most_similar(positive=['友人'], negative=['友情'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('知人', 0.4586910605430603),
 ('自宅', 0.35488438606262207),
 ('知り合い', 0.329221248626709),
 ('入り浸り', 0.3212822675704956),
 ('親戚', 0.31865695118904114),
 ('知り合っ', 0.3158203959465027),
 ('宅', 0.31503963470458984),
 ('誘わ', 0.302945077419281),
 ('繁く', 0.30250048637390137),
 ('同僚', 0.29792869091033936)]

In [0]:
!pip install janome

     |████████████████████████████████| 21.5MB 156kB/s 


In [0]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

In [0]:
import re
import unicodedata
import string


def format_text(text):
    text = unicodedata.normalize("NFKC", text)
    table = str.maketrans("", "", string.punctuation  + "「」、。・")
    text = text.translate(table)

    return text


def preprocessing(text):
    """
    前処理をする関数
    """
    text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))  # 全角→半角

    text = text.lower()  # 大文字→小文字

    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('　', '', text)

    text = re.sub(r'[0-9 ０-９]', '', text)  # 数字の除去
    text = re.sub(r'[!-/:-@[-`{-~]', '', text)  # 半角記号の除去
    text = re.sub(r'/[！-／：-＠［-｀｛-～、-〜”’・]', '', text)  # 全角記号の除去
    text = format_text(text)  # 全角記号の除去

    return  text


def tokenizer_with_preprocessing(text):
    text = preprocessing(text)
    text = tokenizer_janome(text)

    return text

In [0]:
text = '今回は　fastTextを　使って 分散表現を獲得したい！！！?'
print(tokenizer_with_preprocessing(text))

['今回', 'は', 'fasttext', 'を', '使っ', 'て', '分散', '表現', 'を', '獲得', 'し', 'たい']


In [0]:
import torchtext

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True,
                            batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='/content/drive/My Drive/data/tsv', train='train.tsv',
    validation='val.tsv', test='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

In [0]:
from torchtext.vocab import Vectors

vectors = Vectors(name='./vector_neologd/model.vec')

# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

100%|█████████▉| 350609/351122 [00:32<00:00, 10107.70it/s]

torch.Size([49, 300])


defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<pad>': 1,
             '<unk>': 0,
             'い': 16,
             'いる': 9,
             'か': 10,
             'から': 17,
             'が': 6,
             'し': 7,
             'する': 18,
             'その': 19,
             'た': 20,
             'て': 11,
             'で': 21,
             'です': 22,
             'と': 2,
             'な': 3,
             'に': 23,
             'に対して': 24,
             'の': 4,
             'は': 25,
             'まし': 26,
             'ます': 12,
             'を': 8,
             'クラス': 27,
             'ネガティブ': 28,
             'ポジティブ': 29,
             'モデル': 30,
             'レビュー': 31,
             '値': 32,
             '処理': 33,
             '分類': 13,
             '取り組み': 34,
             '商品': 35,
             '女性': 36,
             '女王': 37,
             '好き': 38,
             '姫': 39,
             '学習': 40,
             '文章': 5,
             '本章': 14,
             '構築': 41,
   

In [0]:
import torch.nn.functional as F

tensor_calc = TEXT.vocab.vectors[TEXT.vocab.stoi['単語']]

# コサイン類似度
print("論文", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[TEXT.vocab.stoi['論文']], dim=0))
print("ワード", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[TEXT.vocab.stoi['ワード']], dim=0))
print("ベクトル", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[TEXT.vocab.stoi['ベクトル']], dim=0))

論文 tensor(0.)
ワード tensor(0.)
ベクトル tensor(0.)
